In [1]:
import sqlite3
import pandas as pd

In [2]:
import numpy as np

In [3]:
db = sqlite3.connect("../data/ripe-dns-all-probes-v4v6-extended-resolvers-2020.db")
dff = pd.read_sql_query("SELECT * from data", db)
db.close()

In [4]:
df_reduced = dff[['prb_id', 'resolver_name', 'is_global_ip']]

In [5]:
df_reduced

,prb_id,resolver_name,is_global_ip
0,1,local,0
1,1000,local,0
2,1002,local,0
3,102,local,1
4,102,local,1
...,...,...,...
37449,1000095,Google,1
37450,1000095,Google,1
37451,1000652,local,0
37452,1000022,Cloudflare,1


In [6]:
df_reduced = df_reduced.drop_duplicates()

In [7]:
df_reduced.prb_id.nunique()

10624

In [8]:
df_reduced[df_reduced.resolver_name != 'local'].prb_id.nunique()

3007

In [9]:
df_reduced[df_reduced.resolver_name == 'local'].prb_id.nunique()

9253

In [10]:
doh_autoupgrade = ['CleanBrowsing', 'Cloudflare', 'Google', 'NextDNS', 'OpenDNS', 'Quad9']

In [11]:
df_reduced[df_reduced['resolver_name'].isin(doh_autoupgrade)]['prb_id'].nunique()

2991

In [12]:
probes_with_centr_rslv = df_reduced[df_reduced.resolver_name != 'local'].prb_id.unique()

In [13]:
probes_with_local_rslv = df_reduced[df_reduced.resolver_name == 'local'].prb_id.unique()

In [14]:
#probes with a local resolver and centralized one
df_reduced[(df_reduced.prb_id.isin(probes_with_local_rslv)) & (df_reduced.resolver_name != 'local')].prb_id.nunique()

1636

In [15]:
#sanity check
df_reduced[(df_reduced.prb_id.isin(probes_with_centr_rslv)) & (df_reduced.resolver_name == 'local')].prb_id.nunique()

1636

In [16]:
df_reduced['ctr_local'] = np.where(df_reduced['resolver_name'] == 'local', 1, 0)
df_reduced['ctr_centr'] = np.where(df_reduced['resolver_name'] != 'local', 1, 0)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
df_reduced['ctr_all'] = 1

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
df_reduced[(df_reduced.resolver_name == 'local') & (df_reduced.ctr_centr == 1)]

,prb_id,resolver_name,is_global_ip,ctr_local,ctr_centr,ctr_all


In [19]:
df_reduced[(df_reduced.resolver_name != 'local') & (df_reduced.ctr_local == 1)]

,prb_id,resolver_name,is_global_ip,ctr_local,ctr_centr,ctr_all


In [20]:
df_rslv_ctr = df_reduced.groupby('prb_id').agg({'ctr_local':'sum', 'ctr_centr':'sum', 'ctr_all':'sum'}).reset_index()

In [21]:
print('Overall number of probes')
print(df_rslv_ctr.prb_id.count())
print('Mean number of local resolvers')
print(df_rslv_ctr.ctr_local.mean())
print('Median number of local resolvers')
print(df_rslv_ctr.ctr_local.median())
print('Mean number of centralised resolvers')
print(df_rslv_ctr.ctr_centr.mean())
print('Median number of centralised resolvers')
print(df_rslv_ctr.ctr_centr.median())
print('Mean number of all resolvers')
print(df_rslv_ctr.ctr_all.mean())
print('Median number of all resolvers')
print(df_rslv_ctr.ctr_all.median())

Overall number of probes
10624
Mean number of local resolvers
0.9821159638554217
Median number of local resolvers
1.0
Mean number of centralised resolvers
0.39504894578313254
Median number of centralised resolvers
0.0
Mean number of all resolvers
1.3771649096385543
Median number of all resolvers
1.0


In [22]:
print('Number of probes with only local resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_centr == 0].prb_id.count())
print('Mean number of local resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_centr == 0].ctr_local.mean())
print('Median number of local resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_centr == 0].ctr_local.median())
print('Mean number of centralised resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_centr == 0].ctr_centr.mean())
print('Median number of centralised resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_centr == 0].ctr_centr.median())
print('Mean number of all resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_centr == 0].ctr_all.mean())
print('Median number of all resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_centr == 0].ctr_all.median())

Number of probes with only local resolvers
7617
Mean number of local resolvers
1.1463830904555599
Median number of local resolvers
1.0
Mean number of centralised resolvers
0.0
Median number of centralised resolvers
0.0
Mean number of all resolvers
1.1463830904555599
Median number of all resolvers
1.0


In [23]:
df_rslv_ctr[df_rslv_ctr.ctr_centr == 0].prb_id.count() / df_rslv_ctr.prb_id.count()

0.7169615963855421

In [24]:
#df_rslv_ctr[df_rslv_ctr.ctr_local == 0].prb_id.count()


print('Number of probes with only centralised resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_local == 0].prb_id.count())
print('Mean number of local resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_local == 0].ctr_local.mean())
print('Median number of local resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_local == 0].ctr_local.median())
print('Mean number of centralised resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_local == 0].ctr_centr.mean())
print('Median number of centralised resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_local == 0].ctr_centr.median())
print('Mean number of all resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_local == 0].ctr_all.mean())
print('Median number of all resolvers')
print(df_rslv_ctr[df_rslv_ctr.ctr_local == 0].ctr_all.median())

Number of probes with only centralised resolvers
1371
Mean number of local resolvers
0.0
Median number of local resolvers
0.0
Mean number of centralised resolvers
1.314369073668855
Median number of centralised resolvers
1.0
Mean number of all resolvers
1.314369073668855
Median number of all resolvers
1.0


In [25]:
df_rslv_ctr[df_rslv_ctr.ctr_local == 0].prb_id.count() / df_rslv_ctr.prb_id.count()

0.12904743975903615

In [26]:
#df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].prb_id.count()

print('Number of probes with centralised and local resolvers')
print(df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].prb_id.count())
print('Mean number of local resolvers')
print(df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].ctr_local.mean())
print('Median number of local resolvers')
print(df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].ctr_local.median())
print('Mean number of centralised resolvers')
print(df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].ctr_centr.mean())
print('Median number of centralised resolvers')
print(df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].ctr_centr.median())
print('Mean number of all resolvers')
print(df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].ctr_all.mean())
print('Median number of all resolvers')
print(df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].ctr_all.median())

Number of probes with centralised and local resolvers
1636
Mean number of local resolvers
1.0403422982885087
Median number of local resolvers
1.0
Mean number of centralised resolvers
1.4639364303178484
Median number of centralised resolvers
1.0
Mean number of all resolvers
2.504278728606357
Median number of all resolvers
3.0


In [27]:
df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].prb_id.count() / df_rslv_ctr.prb_id.count()

0.1539909638554217

# Probes with only centralised resolvers

In [28]:
df_only_centr = df_reduced[df_reduced.prb_id.isin(df_rslv_ctr[df_rslv_ctr.ctr_local == 0].prb_id.unique())].copy(deep=True)

In [29]:
df_only_centr

,prb_id,resolver_name,is_global_ip,ctr_local,ctr_centr,ctr_all
25,1083,Google,1,0,1,1
47,1127,Google,1,0,1,1
51,1134,OpenDNS,1,0,1,1
53,1134,Google,1,0,1,1
67,1163,Google,1,0,1,1
...,...,...,...,...,...,...
35419,52633,Google,1,0,1,1
35762,53387,OpenDNS,1,0,1,1
36481,1000081,Cloudflare,1,0,1,1
36936,1000971,Google,1,0,1,1


### overall popularity

In [30]:
df_only_centr.resolver_name.value_counts()

Google           1001
Cloudflare        527
Quad9             126
OpenDNS           122
Yandex             12
NextDNS             8
VeriSign            3
Neustar             2
CleanBrowsing       1
Name: resolver_name, dtype: int64

In [31]:
df_only_centr.resolver_name.value_counts(normalize=True)

Google           0.555494
Cloudflare       0.292453
Quad9            0.069922
OpenDNS          0.067703
Yandex           0.006659
NextDNS          0.004440
VeriSign         0.001665
Neustar          0.001110
CleanBrowsing    0.000555
Name: resolver_name, dtype: float64

### number of resolvers

In [32]:
df_rslv_ctr[df_rslv_ctr.ctr_local == 0].ctr_centr.value_counts()

1    978
2    355
3     38
Name: ctr_centr, dtype: int64

In [33]:
df_rslv_ctr[df_rslv_ctr.ctr_local == 0].ctr_centr.value_counts(normalize=True)

1    0.713348
2    0.258935
3    0.027717
Name: ctr_centr, dtype: float64

In [34]:
probe_dict_only_centr = {}
for ctr_i in df_rslv_ctr[df_rslv_ctr.ctr_local == 0].ctr_centr.unique():
    probe_dict_only_centr[ctr_i] = df_rslv_ctr[(df_rslv_ctr.ctr_local == 0) & (df_rslv_ctr.ctr_centr == ctr_i)].prb_id.unique()

### popularity of resolvers based on number of resolvers

In [35]:
for num_centr_rslv in probe_dict_only_centr.keys():
    print('Probes with '+str(num_centr_rslv)+' centralised resolver(s):')
    print(df_reduced[df_reduced.prb_id.isin(probe_dict_only_centr[num_centr_rslv])].resolver_name.value_counts())
    print('')
    print(df_reduced[df_reduced.prb_id.isin(probe_dict_only_centr[num_centr_rslv])].resolver_name.value_counts(normalize=True))
    print('')

Probes with 1 centralised resolver(s):
Google        662
Cloudflare    197
OpenDNS        81
Quad9          28
NextDNS         7
VeriSign        1
Neustar         1
Yandex          1
Name: resolver_name, dtype: int64

Google        0.676892
Cloudflare    0.201431
OpenDNS       0.082822
Quad9         0.028630
NextDNS       0.007157
VeriSign      0.001022
Neustar       0.001022
Yandex        0.001022
Name: resolver_name, dtype: float64

Probes with 2 centralised resolver(s):
Google           303
Cloudflare       294
Quad9             71
OpenDNS           33
Yandex             5
VeriSign           1
Neustar            1
CleanBrowsing      1
NextDNS            1
Name: resolver_name, dtype: int64

Google           0.426761
Cloudflare       0.414085
Quad9            0.100000
OpenDNS          0.046479
Yandex           0.007042
VeriSign         0.001408
Neustar          0.001408
CleanBrowsing    0.001408
NextDNS          0.001408
Name: resolver_name, dtype: float64

Probes with 3 centralised r

### most popular combinations of resolvers

In [36]:
df_only_centr.groupby('prb_id').agg({'resolver_name':(lambda x: ', '.join(sorted(set(x))))}).resolver_name.value_counts()

Google                         662
Cloudflare, Google             245
Cloudflare                     197
OpenDNS                         81
Cloudflare, Quad9               41
Google, Quad9                   28
Quad9                           28
Cloudflare, Google, Quad9       24
Google, OpenDNS                 24
NextDNS                          7
Cloudflare, OpenDNS              7
Cloudflare, Google, OpenDNS      5
Cloudflare, Google, Yandex       5
Google, Yandex                   4
Cloudflare, OpenDNS, Quad9       2
OpenDNS, Quad9                   2
Yandex                           1
Google, OpenDNS, Yandex          1
CleanBrowsing, NextDNS           1
Neustar                          1
Cloudflare, Yandex               1
Google, Neustar                  1
Google, Quad9, VeriSign          1
Google, VeriSign                 1
VeriSign                         1
Name: resolver_name, dtype: int64

In [37]:
df_only_centr.groupby('prb_id').agg({'resolver_name':(lambda x: ', '.join(sorted(set(x))))}).resolver_name.value_counts(normalize=True)

Google                         0.482859
Cloudflare, Google             0.178702
Cloudflare                     0.143691
OpenDNS                        0.059081
Cloudflare, Quad9              0.029905
Google, Quad9                  0.020423
Quad9                          0.020423
Cloudflare, Google, Quad9      0.017505
Google, OpenDNS                0.017505
NextDNS                        0.005106
Cloudflare, OpenDNS            0.005106
Cloudflare, Google, OpenDNS    0.003647
Cloudflare, Google, Yandex     0.003647
Google, Yandex                 0.002918
Cloudflare, OpenDNS, Quad9     0.001459
OpenDNS, Quad9                 0.001459
Yandex                         0.000729
Google, OpenDNS, Yandex        0.000729
CleanBrowsing, NextDNS         0.000729
Neustar                        0.000729
Cloudflare, Yandex             0.000729
Google, Neustar                0.000729
Google, Quad9, VeriSign        0.000729
Google, VeriSign               0.000729
VeriSign                       0.000729


# Probes with local and centralised resolvers

In [38]:
df_local_centr = df_reduced[df_reduced.prb_id.isin(df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].prb_id.unique())].copy(deep=True)

### overall popularity

In [39]:
df_local_centr.resolver_name.value_counts()

local         1702
Google        1357
VeriSign       656
Cloudflare     263
OpenDNS         54
Quad9           47
Yandex          13
NextDNS          2
Neustar          2
OpenNIC          1
Name: resolver_name, dtype: int64

In [40]:
df_local_centr[df_local_centr['resolver_name'] != 'local']['resolver_name'].value_counts(normalize=True)

Google        0.566597
VeriSign      0.273904
Cloudflare    0.109812
OpenDNS       0.022547
Quad9         0.019624
Yandex        0.005428
NextDNS       0.000835
Neustar       0.000835
OpenNIC       0.000418
Name: resolver_name, dtype: float64

In [41]:
df_local_centr.resolver_name.value_counts(normalize=True)

local         0.415426
Google        0.331218
VeriSign      0.160117
Cloudflare    0.064193
OpenDNS       0.013180
Quad9         0.011472
Yandex        0.003173
NextDNS       0.000488
Neustar       0.000488
OpenNIC       0.000244
Name: resolver_name, dtype: float64

### number of resolvers

In [42]:
df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].ctr_all.value_counts()

3    825
2    811
Name: ctr_all, dtype: int64

In [43]:
df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].ctr_all.value_counts(normalize=True)

3    0.504279
2    0.495721
Name: ctr_all, dtype: float64

In [44]:
probe_dict_local_centr = {}
for ctr_i in df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0)].ctr_all.unique():
    probe_dict_local_centr[ctr_i] = df_rslv_ctr[(df_rslv_ctr.ctr_local != 0) & (df_rslv_ctr.ctr_centr != 0) & (df_rslv_ctr.ctr_all == ctr_i)].prb_id.unique()
    
    
    
    

### popularity of resolvers based on number of resolvers

In [45]:
for num_centr_rslv in probe_dict_local_centr.keys():
    print('Probes with '+str(num_centr_rslv)+' centralised resolver(s):')
    print(df_reduced[df_reduced.prb_id.isin(probe_dict_local_centr[num_centr_rslv])].resolver_name.value_counts())
    print('')
    print(df_reduced[df_reduced.prb_id.isin(probe_dict_local_centr[num_centr_rslv])].resolver_name.value_counts(normalize=True))
    print('')

Probes with 2 centralised resolver(s):
local         811
Google        564
Cloudflare    162
OpenDNS        42
Quad9          28
Yandex         11
NextDNS         2
Neustar         2
Name: resolver_name, dtype: int64

local         0.500000
Google        0.347719
Cloudflare    0.099877
OpenDNS       0.025894
Quad9         0.017263
Yandex        0.006782
NextDNS       0.001233
Neustar       0.001233
Name: resolver_name, dtype: float64

Probes with 3 centralised resolver(s):
local         891
Google        793
VeriSign      656
Cloudflare    101
Quad9          19
OpenDNS        12
Yandex          2
OpenNIC         1
Name: resolver_name, dtype: int64

local         0.360000
Google        0.320404
VeriSign      0.265051
Cloudflare    0.040808
Quad9         0.007677
OpenDNS       0.004848
Yandex        0.000808
OpenNIC       0.000404
Name: resolver_name, dtype: float64



### most popular combinations of resolvers

In [46]:
df_local_centr.groupby('prb_id').agg({'resolver_name':(lambda x: ', '.join(sorted(set(x))))}).resolver_name.value_counts()



Google, VeriSign, local       656
Google, local                 611
Cloudflare, local             175
Cloudflare, Google, local      75
OpenDNS, local                 44
Quad9, local                   32
Yandex, local                  11
Cloudflare, Quad9, local       10
Google, OpenDNS, local          9
Google, Quad9, local            5
Neustar, local                  2
NextDNS, local                  2
Cloudflare, OpenDNS, local      1
Cloudflare, Yandex, local       1
Google, Yandex, local           1
Cloudflare, OpenNIC, local      1
Name: resolver_name, dtype: int64

In [47]:
df_local_centr.groupby('prb_id').agg({'resolver_name':(lambda x: ', '.join(sorted(set(x))))}).resolver_name.value_counts(normalize=True)




Google, VeriSign, local       0.400978
Google, local                 0.373472
Cloudflare, local             0.106968
Cloudflare, Google, local     0.045844
OpenDNS, local                0.026895
Quad9, local                  0.019560
Yandex, local                 0.006724
Cloudflare, Quad9, local      0.006112
Google, OpenDNS, local        0.005501
Google, Quad9, local          0.003056
Neustar, local                0.001222
NextDNS, local                0.001222
Cloudflare, OpenDNS, local    0.000611
Cloudflare, Yandex, local     0.000611
Google, Yandex, local         0.000611
Cloudflare, OpenNIC, local    0.000611
Name: resolver_name, dtype: float64